# Reduce GIWAXS Transformation

In [ ]:
from pathlib import Path
import pyFAI
import fabio
import numpy as np
import yaml
import matplotlib.pylab as plt
#%matplotlib widget  # need `pip install ipympl` and `jupyter nbextension enable --py widgetsnbextension
from matplotlib.colors import LogNorm
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

title = ""  # adds this to the title of the plots

IM_SIZE = (6.3, 3)  # inches

raw_stitch = fabio.open("image-transformed.tif").data.astype(np.float64)
flat_field = fabio.open("flat-field-transformed.tif").data.astype(np.float64)
with open("params.yaml", "r") as yf:
    params = yaml.safe_load(yf)
    exposure_time = params["exposure"]
    incident_angle = params["incident"]
    try:
        tilt_angle = params["tilt"]
    except KeyError:
        tilt_angle = 0
if tilt_angle is None:
    titl_angle = 0
print(f"Exposure time  = {exposure_time} s / (2 images)")
print(f"incident angle = {incident_angle} degrees")
print(f"tilt angle     = {tilt_angle} degrees")

print('Generated "fake" flat field from exposure time at each pixel')

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(raw_stitch, norm=LogNorm(1, np.max(raw_stitch)))
ax.set_title("Transformed image")
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7, label="counts")
fig.tight_layout()

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(flat_field * exposure_time)
ax.set_title("Exposure time per pixel")
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7, label="exposure time (s)")
fig.tight_layout()

## Adust pixel weights for presentation of stitch

In [ ]:
adjusted_stitch = raw_stitch / flat_field
adjusted_stitch[np.where(adjusted_stitch == np.infty)] = 0
adjusted_stitch = np.nan_to_num(adjusted_stitch)
print(adjusted_stitch.max())

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(adjusted_stitch, norm=LogNorm(1, np.max(adjusted_stitch)))
fig_title = "Transformed image with flat field correction"
if title:
    fig_title = title + "\n" + fig_title
ax.set_title(fig_title)
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7)
fig.tight_layout()
fig.savefig("transformed_image.png", dpi=900, bbox_inches="tight")

## Create Cake and reduce data
You must have a `cal-transformed.poni` file. This should have automatically been generated

You can create a mask using pyFAI-calib2 GUI

In [ ]:
ai = pyFAI.load("cal-transformed.poni")
print("Loaded geometry:")
print(ai)
print("")

try:
    mask = fabio.open("mask.edf").data.astype(bool)
    print("Used mask")
except FileNotFoundError:
    mask = np.zeros(raw_stitch.shape)
    print("Did not load mask, you can create one using pyFAI-calib2")

"""Mask pixels that have 0 in the flat field"""
mask = np.logical_or(mask, np.logical_not(flat_field))

#### Create Cake

In [ ]:
q_bins = 500
azimuthal_bins = 180

chi_0 = 270     # this is where the azimuthal chi is 0

chi_range = (-90, 90)
q_range = None

cake = ai.integrate2d_ng(
    raw_stitch, q_bins, azimuthal_bins,
    radial_range=q_range,          # In units specified below
    azimuth_range=(chi_range[0] + chi_0 , chi_range[1] + chi_0),   # this is 0 to 180 from the right going counter-clockwise
    mask=mask, flat=flat_field,
    error_model="poisson",  unit="q_A^-1",
    polarization_factor=None, correctSolidAngle=False,
)

fig, ax = plt.subplots(1, 1, figsize=(4, 3), facecolor="w")
pos = ax.imshow(cake[0], norm=LogNorm(1, np.max(cake[0])),
                 extent=(np.min(cake[1]), np.max(cake[1]), *chi_range),
                 aspect='auto')
fig_title = "Cake"
if title:
    fig_title = title + "\n" + fig_title
ax.set_title(fig_title)
ax.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax.set_ylabel(r"$\psi (\degree)$")
ax.set_yticks(np.arange(*chi_range, 20))
ax.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)
# ax.set_xticks(np.arange(0, np.min(cake[1]) + np.max(cake[1]), 0.5))
ax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
fig.colorbar(pos, ax=ax)
fig.tight_layout()
fig.savefig(f"{title}-cake.png".lstrip("-"), dpi=900, bbox_inches="tight")

#### Create Reduction

In [ ]:
q_range = None
q_bins = 1000
file_to_save = f"{title}-reduction.edf".lstrip("-")
if (Path.cwd() / file_to_save).is_file():
    (Path.cwd() / file_to_save).unlink()

redu = ai.integrate1d_ng(
    raw_stitch, q_bins,
    radial_range=q_range,          # In units specified below
    azimuth_range=(180, 360),   # this is 0 to 180 from the right going counter-clockwise
    mask=mask, flat=flat_field, error_model="poisson",
    unit="q_A^-1", filename=f"{title}-reduction.edf".lstrip("-"), normalization_factor=(exposure_time / 60)
)

fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.scatter(
    redu[0], redu[1],
    s=5,  # marker size
    marker="o",  # marker shape
    edgecolors="black",  # marker edge color
    lw=.75,  # marker edge width
    alpha=1,  # transparency
    facecolor='w'  # marker face color
)

fig_title = "Reduction"
if title:
    fig_title = title + "\n" + fig_title
ax.set_title(fig_title)
# ax.set_xticks(np.arange(0, np.min(redu[0]) + np.max(redu[0]), 0.5))
ax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
ax.tick_params(axis='both', which='both', direction='in', right=True, top=True)
ax.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax.grid(linestyle='dotted')
ax.set_ylabel(r"Intensity rate (counts / (min $\cdot$ projected pixel area)")
ax.set_yscale("log")
fig.savefig(f"{title}-reduction.png".lstrip("-"), dpi=1000, bbox_inches="tight")

#### Pole Plot

In [ ]:
ring_name = ""

bins = 720
q_range = None      # set around desired ring
azi_range = None    # must be -60 to 60 for local specular

if azi_range is None:
    azi_range = (chi_0 - 90, chi_0 + 90)
else:
    azi_range = (chi_0 + azi_range[0], chi_0 + azi_range[1])

pole = ai.integrate_radial(
    raw_stitch, bins,
    radial_range=q_range, 
    azimuth_range=azi_range,
    mask=mask, flat=flat_field,
    radial_unit="q_A^-1"
)

chi = pole[0] + 90
pole_counts = pole[1]

fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.scatter(
    chi, pole_counts,
    s=5,  # marker size
    marker="o",  # marker shape
    edgecolors="black",  # marker edge color
    lw=.75,  # marker edge width
    alpha=1,  # transparency
    facecolor='w'  # marker face color
)

ax.set_title("Reduction")
ax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
# ax.set_xticks(np.arange(0, np.min(redu[0]) + np.max(redu[0]), 0.5))
ax.set_xlabel(r"$\psi\ (\degree)$")
ax.grid(linestyle='dotted')
ax.set_ylabel("counts / pixel")
ax.set_yscale("log")
ax.tick_params(axis='both', which='both', direction='in', right=True, top=True)
# fig.savefig(f"pole-{ring_name}.png", dpi=900, bbox_inches="tight")